In [1]:
#import os, random, numpy as np, tensorflow as tf
#os.environ["PYTHONHASHSEED"]="137";
#random.seed(137);
#np.random.seed(137);
#tf.random.set_seed(137)


2025-09-03 22:50:00.812238: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-03 22:50:02.463508: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-03 22:50:02.463574: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-03 22:50:02.470931: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-03 22:50:03.439508: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-03 22:50:07.503792: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT

In [1]:
import sys, os, platform, tensorflow as tf
print("Python:", platform.python_version())
print("Executable:", sys.executable)
print("Conda prefix:", os.environ.get("CONDA_PREFIX"))
print("TF:", tf.__version__)

2025-09-03 22:58:27.095669: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-03 22:58:27.159910: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-03 22:58:27.159957: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-03 22:58:27.160003: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-03 22:58:27.171184: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-03 22:58:28.940826: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT

Python: 3.10.18
Executable: /home/bballdave025/miniforge3/envs/vanilla_cnn/bin/python
Conda prefix: /home/bballdave025/miniforge3/envs/vanilla_cnn
TF: 2.14.0


In [2]:
import os, random, numpy as np, tensorflow as tf
os.environ["PYTHONHASHSEED"]="137";
random.seed(137);
np.random.seed(137);
tf.random.set_seed(137)

In [5]:
# --- CIFAR-10 cache + quieter TF (paste at very top) ---
import os, shutil
from pathlib import Path

# Quieter + modest threads (optional)
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")
os.environ.setdefault("OMP_NUM_THREADS", "4")
os.environ.setdefault("TF_NUM_INTEROP_THREADS", "2")
os.environ.setdefault("TF_NUM_INTRAOP_THREADS", "4")

# Use project-local cache under $TAGDIR/datasets; fall back to ~/.keras/datasets if present
tagdir = Path(os.environ.get("TAGDIR", Path.cwd()))
if tagdir.name == "notebooks":
    tagdir = tagdir.parent
os.environ.setdefault("KERAS_HOME", str(tagdir))  # Keras will use $KERAS_HOME/datasets
proj_cache = Path(os.environ["KERAS_HOME"]) / "datasets"
proj_cache.mkdir(parents=True, exist_ok=True)

home_cache = Path.home() / ".keras" / "datasets"
# If project cache lacks CIFAR but home cache has it, mirror it over:
candidates = [("cifar-10-python.tar.gz", True), ("cifar-10-batches-py", False)]
if home_cache.exists() and not any((proj_cache / name).exists() for name, _ in candidates):
    for name, is_file in candidates:
        src = home_cache / name
        if src.exists():
            dst = proj_cache / src.name
            if is_file:
                shutil.copy2(src, dst)
            else:
                shutil.copytree(src, dst, dirs_exist_ok=True)

# Import TF only after env vars are set
import tensorflow as tf
tf.config.threading.set_intra_op_parallelism_threads(int(os.environ["TF_NUM_INTRAOP_THREADS"]))
tf.config.threading.set_inter_op_parallelism_threads(int(os.environ["TF_NUM_INTEROP_THREADS"]))



######## From first try, still keep #############

# Minimal CIFAR-10 Q&R cell — seed=137, CPU, saves CSV + JSON + prints [DONE]
import os, json, time, random, csv
from pathlib import Path
import numpy as np, tensorflow as tf

# Force CPU (optional; harmless if no GPU)
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "")

# Repro-ish seed
SEED = 137
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

# Resolve TAGDIR robustly
tagdir_env = os.environ.get("TAGDIR", "")
tagdir = Path(tagdir_env) if tagdir_env else Path.cwd()
if tagdir.name == "notebooks":  # launched notebook inside TAGDIR/notebooks
    tagdir = tagdir.parent

# Outputs
out_dir = tagdir / "outputs"
csv_dir = out_dir / "csv_logs"
csv_dir.mkdir(parents=True, exist_ok=True)
ts = time.strftime("%Y%m%dT%H%M%S")

# Data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test  = tf.keras.utils.to_categorical(y_test, 10)

# Tiny CNN (fast on CPU)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation="relu", padding="same", input_shape=(32,32,3)),
    tf.keras.layers.Conv2D(32, 3, activation="relu"),
    tf.keras.layers.MaxPooling2D(), tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same"),
    tf.keras.layers.Conv2D(64, 3, activation="relu"),
    tf.keras.layers.MaxPooling2D(), tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"), tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation="softmax"),
])
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

hist = model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.1, verbose=2)

# Test + artifacts
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

csv_path = csv_dir / f"train_history_seed{SEED}_{ts}.csv"
with open(csv_path, "w", newline="") as f:
    keys = list(hist.history.keys())
    writer = csv.writer(f); writer.writerow(["epoch"] + keys)
    for i in range(len(hist.history[keys[0]])):
        writer.writerow([i+1] + [hist.history[k][i] for k in keys])

summary_path = out_dir / f"test_summary_seed{SEED}_{ts}.json"
with open(summary_path, "w") as f:
    json.dump({"seed": SEED, "timestamp": ts, "test_accuracy": float(test_acc)}, f, indent=2)

print(f"[DONE] test_acc={test_acc:.4f}")

RuntimeError: Intra op parallelism cannot be modified after initialization.